# O intuito desse notebook e recuperar todas as imagens disponiveis  sobre pneumonia e covid e disponibilizar em um banco de dados


## bibliotecas necessárias

In [1]:
!pip install imutils

In [1]:
from imutils import paths
from PIL import Image
import pydicom as dicom
import pandas as pd
import shutil
import cv2 
import os

##  diretorios target e source

In [2]:
def getTargetDiretorio(tipo):
    retorno = ""
    if(tipo =="MaquinaLocal"):
        retorno = r'D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto'

    if(tipo =="GoogleDrive"):
       retorno=r'/content/drive/My Drive/TCC Pos Graduação big data/TCC/Projeto/'
       
    
    return retorno

In [3]:
dir_target_diretorio_raiz = getTargetDiretorio("MaquinaLocal")
dir_target_diretorio_dataset =r"{0}/{1}".format(dir_target_diretorio_raiz,'dataset')
dir_target_diretorio_imagem_covid =r"{0}/{1}".format(dir_target_diretorio_dataset,'covid')
dir_target_diretorio_pneumonia_normal =r"{0}/{1}".format(dir_target_diretorio_dataset,'normal')
dir_target_diretorio_pneumonia_pneumonia =r"{0}/{1}".format(dir_target_diretorio_dataset,'pneumonia')
dir_target_diretorio_validacao =r"{0}/{1}".format(dir_target_diretorio_dataset,'validacao')



dir_source_covid_cohen =r'D:/Repos/datasetsGitHub/covid-chestxray-dataset'
dir_source_covid_figure1 =r'D:/Repos/datasetsGitHub/Figure1-COVID-chestxray-dataset'
dir_source_covid_Actualmed =r'D:/Repos/datasetsGitHub/Actualmed-COVID-chestxray-dataset'
dir_source_RadiographyDatabase =r'D:/Repos/datasetsGitHub/COVID-19 Radiography Database'

dir_source_rsna_pneumonia =r'D:/Repos/rsna-pneumonia-detection-challenge'


print("source")
print(dir_source_covid_cohen)
print(dir_source_covid_figure1)
print(dir_source_covid_Actualmed)
print(dir_source_RadiographyDatabase)

print("target")
print(dir_target_diretorio_raiz)
print(dir_target_diretorio_dataset)
print(dir_target_diretorio_imagem_covid)
print(dir_target_diretorio_pneumonia_normal)
print(dir_target_diretorio_pneumonia_pneumonia)
print(dir_target_diretorio_validacao)







source
D:/Repos/datasetsGitHub/covid-chestxray-dataset
D:/Repos/datasetsGitHub/Figure1-COVID-chestxray-dataset
D:/Repos/datasetsGitHub/Actualmed-COVID-chestxray-dataset
D:/Repos/datasetsGitHub/COVID-19 Radiography Database
target
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset/covid
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset/normal
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset/pneumonia
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset/validacao


In [4]:
lista_diretorios_target=[]
lista_diretorios_target.append(dir_target_diretorio_raiz)
lista_diretorios_target.append(dir_target_diretorio_dataset),
lista_diretorios_target.append(dir_target_diretorio_imagem_covid),
lista_diretorios_target.append(dir_target_diretorio_pneumonia_normal),
lista_diretorios_target.append(dir_target_diretorio_pneumonia_pneumonia)
lista_diretorios_target.append(dir_target_diretorio_validacao)
lista_diretorios_target.append(r"{0}/{1}".format(dir_target_diretorio_validacao,"covid"))
lista_diretorios_target.append(r"{0}/{1}".format(dir_target_diretorio_validacao,"normal"))
lista_diretorios_target.append(r"{0}/{1}".format(dir_target_diretorio_validacao,"pneumonia"))

for item in lista_diretorios_target:
    if not os.path.exists(item):
        os.mkdir(item)
    




## metodos auxiliares

In [5]:
def clear_diretorio(diretorio):
    for file in os.listdir(diretorio):
        full_name =os.path.join(diretorio, file)
        
        if os.path.exists(full_name):
            os.remove(full_name) 
        
        

def move_imagens_from_source_to_target(source_lista,dir_target):
    for source_path in source_lista:
        target_imagem =""
        imagemName =source_path.split('/')[-1]
        if(imagemName.count(".dcm") > 0):
            target_imagem ="{0}/{1}".format(dir_target,imagemName.replace(".dcm",".jpg"))
            if not os.path.exists(target_imagem):
                ds = dicom.dcmread(source_path)
                pixel_array_numpy = ds.pixel_array
                cv2.imwrite(target_imagem, pixel_array_numpy)
            
        else:
            target_imagem ="{0}/{1}".format(dir_target,imagemName)
            if not os.path.exists(target_imagem):
                shutil.copy2(source_path,target_imagem)
                




## Leitura dos Datasets

In [6]:

## repositorios locais
source_dir_metadata_local_cohen  = r"{0}/{1}".format(dir_source_covid_cohen,'metadata.csv')
dir_source_imagens_Covid_cohen = r"{0}/{1}".format(dir_source_covid_cohen,'images')

source_dir_metadata_local_figure_1  = r"{0}/{1}".format(dir_source_covid_figure1,'metadata.csv')
source_dir_imagens_figure_1  = r"{0}/{1}".format(dir_source_covid_figure1,'images')

source_dir_metadata_local_Actualmed  = r"{0}/{1}".format(dir_source_covid_Actualmed,'metadata.csv')
source_dir_imagens_Actualmed = r"{0}/{1}".format(dir_source_covid_Actualmed,'images')



## diretorios pra pneumonia

dir_source_pneumonia_csv_train_detailed = r"{0}/{1}".format(dir_source_rsna_pneumonia,'stage_2_detailed_class_info.csv') 
dir_source_pneumonia_detailed_class_imagens =r"{0}/{1}".format(dir_source_rsna_pneumonia,'stage_2_train_images')




## limpa os diretorios destino das imagens

In [28]:


# if(lista_diretorios_target.count(r'D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto') > 0):
#     lista_diretorios_target.remove(r'D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto')

# if(lista_diretorios_target.count(r'D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset') > 0):
#     lista_diretorios_target.remove(r'D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset')

# if(lista_diretorios_target.count(r'D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset/validacao') > 0):
#     lista_diretorios_target.remove(r'D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset/validacao')



# for diretorio in lista_diretorios_target:
#     clear_diretorio(diretorio)     
    
 
   

In [7]:
map_prognostico  =  { 'Lung Opacity': 'pneumonia',
    'Normal' :'normal',
    'COVID-19':'covid-19',
    'SARS': 'pneumonia',
    'MERS': 'pneumonia',
    'Streptococcus': 'pneumonia',
    'Klebsiella': 'pneumonia',
    'Chlamydophila': 'pneumonia',
    'Legionella': 'pneumonia',
    'Normal': 'normal',
    'Pneumonia' : 'pneumonia'

}

In [8]:


# df_local_cohen = pd.read_csv(source_dir_metadata_local_cohen,error_bad_lines=False)
df_local_figure_1 = pd.read_csv(source_dir_metadata_local_figure_1,error_bad_lines=False,encoding='ISO-8859-1')
df_local_cohen = pd.read_csv(source_dir_metadata_local_cohen,error_bad_lines=False,encoding='ISO-8859-1')
df_local_Actualmed = pd.read_csv(source_dir_metadata_local_Actualmed,error_bad_lines=False,encoding='ISO-8859-1')

df_local_RadiographyDatabase_Covid = pd.read_excel(r"{0}/{1}".format(dir_source_RadiographyDatabase,'COVID-19.metadata.xlsx'),error_bad_lines=False, encoding='ISO-8859-1')

df_local_RadiographyDatabase_Normal = pd.read_excel(r"{0}/{1}".format(dir_source_RadiographyDatabase,'NORMAL.metadata.xlsx'),error_bad_lines=False, encoding='ISO-8859-1')

df_local_RadiographyDatabase_Pneumonia = pd.read_excel(r"{0}/{1}".format(dir_source_RadiographyDatabase,'Viral Pneumonia.matadata.xlsx'),error_bad_lines=False, encoding='ISO-8859-1')


source_dir_metadata_RadiographyDatabase_Imagens_Covid = r"{0}/{1}".format(dir_source_RadiographyDatabase,'COVID-19')
source_dir_metadata_RadiographyDatabase_Imagens_Normal = r"{0}/{1}".format(dir_source_RadiographyDatabase,'NORMAL')
source_dir_metadata_RadiographyDatabase_Imagens_Pneumonia= r"{0}/{1}".format(dir_source_RadiographyDatabase,'Viral Pneumonia')



df_rsna = pd.read_csv(dir_source_pneumonia_csv_train_detailed,error_bad_lines=False,encoding='ISO-8859-1')

print("Quantidade registros figure_1:{0}".format(df_local_figure_1.shape))
print("Quantidade registros cohen:{0}".format(df_local_cohen.shape))
print("Quantidade registros Actualmed:{0}".format(df_local_Actualmed.shape))
print("Quantidade registros RadiographyDatabase Covid:{0}".format(df_local_RadiographyDatabase_Covid.shape))
print("Quantidade registros RadiographyDatabase Normal:{0}".format(df_local_RadiographyDatabase_Normal.shape))
print("Quantidade registros RadiographyDatabase Pneunonia:{0}".format(df_local_RadiographyDatabase_Pneumonia.shape))
print("Quantidade registros Actualmed:{0}".format(df_local_Actualmed.shape))
print("Quantidade registros Rsna:{0}".format(df_rsna.shape))

Quantidade registros figure_1:(55, 12)
Quantidade registros cohen:(682, 29)
Quantidade registros Actualmed:(238, 13)
Quantidade registros RadiographyDatabase Covid:(219, 4)
Quantidade registros RadiographyDatabase Normal:(1341, 4)
Quantidade registros RadiographyDatabase Pneunonia:(1345, 4)
Quantidade registros Actualmed:(238, 13)
Quantidade registros Rsna:(30227, 2)


## Valida a quantidade de registros locais e web


In [9]:
metadata_web_git_cohen ='https://raw.githubusercontent.com/ieee8023/covid-chestxray-dataset/master/metadata.csv'

metadata_web_git_figure_1  ='https://raw.githubusercontent.com/agchung/Figure1-COVID-chestxray-dataset/master/metadata.csv'

metadata_web_git_Actualmed  ='https://raw.githubusercontent.com/agchung/Actualmed-COVID-chestxray-dataset/master/metadata.csv'

In [10]:
df_web_cohen = pd.read_csv(metadata_web_git_cohen,error_bad_lines=False,encoding='ISO-8859-1')
df_web_figure_1 = pd.read_csv(metadata_web_git_figure_1,error_bad_lines=False,encoding='ISO-8859-1')
df_web_Actualmed = pd.read_csv(metadata_web_git_Actualmed,error_bad_lines=False,encoding='ISO-8859-1')

qtd_diferenca_cohen = len(df_web_cohen) - len(df_local_cohen)
qtd_diferenca_figure_1 = len(df_web_figure_1) - len(df_local_figure_1)
qtd_diferenca_Actualmed = len(df_web_Actualmed) - len(df_local_Actualmed)

if(qtd_diferenca_cohen < 0):
    print("Foram removidas {0} : imagens no repositorio web Chohen".format(qtd_diferenca_cohen.__abs__()))
else:
    print("Foram adicionadas {0} : imagens no repositorio web Chohen".format(qtd_diferenca_cohen.__abs__()))


if(qtd_diferenca_figure_1 < 0):
    print("Foram removidas {0} : imagens no repositorio web Figure 1".format(qtd_diferenca_figure_1.__abs__()))
else:
    print("Foram adicionadas {0} : imagens no repositorio web Figure 1".format(qtd_diferenca_figure_1.__abs__()))    

if(qtd_diferenca_Actualmed < 0):
    print("Foram removidas {0} : imagens no repositorio web Actualmed".format(qtd_diferenca_Actualmed.__abs__()))
else:
    print("Foram adicionadas {0} : imagens no repositorio web Actualmed".format(qtd_diferenca_Actualmed.__abs__()))    
        







Foram removidas 9 : imagens no repositorio web Chohen
Foram adicionadas 0 : imagens no repositorio web Figure 1
Foram adicionadas 0 : imagens no repositorio web Actualmed


## tranferencia dos arquivos de Covid 19

In [11]:
df_local_cohen['prognostico'] = df_local_cohen['finding'].map(map_prognostico)


df_local_cohen['prognostico'].value_counts()

# df_covid =  df_metadata.query('indicadorPrognostico == 1 and posicao_imagem =="PA" and tipoExame =="X-ray"')

covid-19     451
pneumonia     63
Name: prognostico, dtype: int64

In [13]:
df_local_figure_1['prognostico'] = df_local_figure_1['finding'].map(map_prognostico)


df_local_figure_1['prognostico'].value_counts()

covid-19     35
pneumonia     2
Name: prognostico, dtype: int64

In [14]:
df_local_Actualmed['prognostico'] = df_local_Actualmed['finding'].map(map_prognostico)

df_local_Actualmed['prognostico'].value_counts()



covid-19    58
Name: prognostico, dtype: int64

In [15]:

df_local_RadiographyDatabase_Covid['prognostico'] ="covid-19"


df_local_RadiographyDatabase_Covid['prognostico'].value_counts()



covid-19    219
Name: prognostico, dtype: int64

In [16]:
df_local_Actualmed['prognostico'] = df_local_Actualmed['finding'].map(map_prognostico)


df_local_Actualmed['prognostico'].value_counts()

covid-19    58
Name: prognostico, dtype: int64

In [17]:


df_rsna['prognostico'] = df_rsna['class'].map(map_prognostico)


df_rsna['prognostico'].value_counts()

pneumonia    9555
normal       8851
Name: prognostico, dtype: int64

In [18]:
lista_imagens_covid =[]
lista_imagens_pneumonia =[]
lista_imagens_normal =[]

quantidade_registros = {'normal': 0, 'pneumonia': 0, 'covid-19': 0}

# diretorio dos dados Covid

## Josephpcohen https://josephpcohen.com/w/public-covid19-dataset/

In [19]:
for row in df_local_cohen[["prognostico","filename"]].iterrows():
    imagem_name = row[1].filename
    prognostico = row[1].prognostico
    source_imagem = "{0}/{1}".format(dir_source_imagens_Covid_cohen,imagem_name)
    if os.path.exists(source_imagem):
        if(prognostico =="covid-19"):
            lista_imagens_covid.append(source_imagem)
            quantidade_registros[prognostico] +=1
        elif (prognostico =="pneumonia"):
            lista_imagens_pneumonia.append(source_imagem)
            quantidade_registros[prognostico] +=1


print(quantidade_registros)
            
            

{'normal': 0, 'pneumonia': 63, 'covid-19': 430}


In [20]:
for row in df_local_figure_1[["prognostico","patientid"]].iterrows():
    imagem_name = row[1].patientid
    prognostico = row[1].prognostico
    source_imagem = "{0}/{1}.jpg".format(source_dir_imagens_figure_1,imagem_name)
    if os.path.exists(source_imagem):
        if(prognostico =="covid-19"):
            lista_imagens_covid.append(source_imagem)
            quantidade_registros[prognostico] +=1
        elif (prognostico =="pneumonia"):
            lista_imagens_pneumonia.append(source_imagem)
            quantidade_registros[prognostico] +=1


print(quantidade_registros)

{'normal': 0, 'pneumonia': 65, 'covid-19': 460}


In [21]:
for row in df_local_Actualmed[["prognostico","imagename"]].iterrows():
    imagem_name = row[1].imagename
    prognostico = row[1].prognostico
    source_imagem = "{0}/{1}".format(source_dir_imagens_Actualmed,imagem_name)
    if os.path.exists(source_imagem):
        if(prognostico =="covid-19"):
            lista_imagens_covid.append(source_imagem)
            quantidade_registros[prognostico] +=1
        elif (prognostico =="pneumonia"):
            lista_imagens_pneumonia.append(source_imagem)
            quantidade_registros[prognostico] +=1


print(quantidade_registros)

{'normal': 0, 'pneumonia': 65, 'covid-19': 518}


In [22]:
for row in df_local_RadiographyDatabase_Covid[["prognostico","FILE NAME","FORMAT"]].iterrows():
    imagem_name ="{0}.{1}".format(row[1]["FILE NAME"],str(row[1]["FORMAT"]).lower())
    prognostico = row[1].prognostico
    source_imagem = "{0}/{1}".format(source_dir_metadata_RadiographyDatabase_Imagens_Covid,imagem_name)
    if os.path.exists(source_imagem):
        if(prognostico =="covid-19"):
            lista_imagens_covid.append(source_imagem)
            quantidade_registros[prognostico] +=1
        elif (prognostico =="pneumonia"):
            lista_imagens_pneumonia.append(source_imagem)
            quantidade_registros[prognostico] +=1

print(quantidade_registros)

{'normal': 0, 'pneumonia': 65, 'covid-19': 604}


In [23]:
df_rsna = df_rsna[(df_rsna.prognostico == "normal") | (df_rsna.prognostico == "pneumonia")]



In [24]:
for row in df_rsna[["patientId","prognostico"]].iterrows():
    imagem_name = row[1].patientId
    prognostico = row[1].prognostico
    source_imagem = "{0}/{1}.dcm".format(dir_source_pneumonia_detailed_class_imagens,imagem_name)
    if os.path.exists(source_imagem):
        dir_target =""
        if(prognostico =="normal"):
            lista_imagens_normal.append(source_imagem)
            quantidade_registros[prognostico] +=1

        elif (prognostico =="pneumonia"):
            lista_imagens_pneumonia.append(source_imagem)
            quantidade_registros[prognostico] +=1

         

print(quantidade_registros)

{'normal': 8851, 'pneumonia': 9620, 'covid-19': 604}


## recupera 50 imagens para cada tipo de classe

In [26]:
import random as rn

rn.seed(1234)
# print(dir_target_diretorio_imagem_covid)
quantidadeImagens = 50
lista_imagens_validacao_covid = []
lista_imagens_validacao_pneumonia = []
lista_imagens_validacao_normal = []

while (len(lista_imagens_validacao_covid) <50 and len(lista_imagens_validacao_pneumonia) <=50 and  len(lista_imagens_validacao_normal) <50):
    if(len(lista_imagens_validacao_covid) <=50):
        imagen_covid = rn.choice(lista_imagens_covid)
        if(lista_imagens_validacao_covid.count(imagen_covid) ==0):
            lista_imagens_validacao_covid.append(imagen_covid)
            if(lista_imagens_covid.count(imagen_covid) > 0):
                lista_imagens_covid.remove(imagen_covid)
            


    if(len(lista_imagens_validacao_pneumonia) <=50):
        imagen_pneumonia = rn.choice(lista_imagens_pneumonia)
        if(lista_imagens_validacao_pneumonia.count(imagen_pneumonia) ==0):
            lista_imagens_validacao_pneumonia.append(imagen_pneumonia)
            

    if(len(lista_imagens_validacao_normal) <=50):
        imagen_normal = rn.choice(lista_imagens_normal)
        if(lista_imagens_validacao_normal.count(imagen_normal) ==0):
            lista_imagens_validacao_normal.append(imagen_normal)
            
    
print("quantidade imagens validação covid :{0}".format(len(lista_imagens_validacao_covid)))
print("quantidade imagens validação pneumonia :{0}".format(len(lista_imagens_validacao_pneumonia)))
print("quantidade imagens validação normal :{0}".format(len(lista_imagens_validacao_normal)))


   
       
       

quantidade imagens validação covid :50
quantidade imagens validação pneumonia :50
quantidade imagens validação normal :50


# Executa a tranferencias das imagens entre os diretorios

In [54]:

## move as imagens de covid
move_imagens_from_source_to_target(lista_imagens_covid,dir_target_diretorio_imagem_covid)
# ## move as imagens normais
# move_imagens_from_source_to_target(lista_imagens_normal,dir_target_diretorio_pneumonia_normal)
# ## move as imagens de pneumonia
# move_imagens_from_source_to_target(lista_imagens_pneumonia,dir_target_diretorio_pneumonia_pneumonia)

# ## move as imagens de validação
# move_imagens_from_source_to_target(lista_imagens_validacao_covid,"{0}/{1}".format(dir_target_diretorio_validacao,"covid"))
# move_imagens_from_source_to_target(lista_imagens_validacao_normal,"{0}/{1}".format(dir_target_diretorio_validacao,"normal"))
# move_imagens_from_source_to_target(lista_imagens_validacao_pneumonia,"{0}/{1}".format(dir_target_diretorio_validacao,"pneumonia"))
 
